In [1]:
# Reload the notebook if an external file is updated
%load_ext autoreload
%autoreload 2

import sys

from pathlib import Path

path = str(Path.cwd().parent)
sys.path.append(path)

In [2]:
import numpy as np

from constant import SETTINGS
from copy import deepcopy
from datatype.dataset import Dataset
from datatype.settings import Settings
from datatype.spectrogram import Linear, Mel, Spectrogram
from io import BytesIO
from IPython.display import display as inline
from PIL import Image, ImageDraw, ImageEnhance, ImageFilter, ImageFont, ImageOps
from wand.image import Image as ImageMagick
from wand.display import display

In [3]:
def create_image(spectrogram):
    buffer = BytesIO()
    image = Image.fromarray(~spectrogram)
    
    return image

In [4]:
def create_spectrogram(signal, settings):
    spectrogram = Spectrogram()
    strategy = Mel(signal, settings)
    spectrogram.strategy = strategy

    return spectrogram.generate()

In [5]:
dataset = Dataset('segment')
dataframe = dataset.load()

In [6]:
path = SETTINGS.joinpath('dereverberate.json')
dereverberate = Settings.from_file(path)

In [7]:
path = SETTINGS.joinpath('spectrogram.json')
settings = Settings.from_file(path)

In [8]:
segment = dataframe.at[45, 'spectrogram']

with ImageMagick.from_array(~segment) as image:
    image.despeckle()
    
    image.opaque_paint(
        target='white', 
        fill='white', 
        fuzz=0.20 * image.quantum_range, 
        invert=False
    )
    
    image.gamma(0.6)
    
    blob = image.make_blob(format='png')
    
    buffer = BytesIO(blob)
    note = Image.open(buffer)
    note.show()